In [1]:
# import the library module for SHA
import sys
import hashlib
import numpy as np
import random

if sys.version_info < (3, 6):
	import sha3

In [2]:
# login to AzureQuantum
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
  resource_id="/subscriptions/b1d7f7f8-743f-458e-b3a0-3e09734d716d/resourceGroups/aq-hackathons/providers/Microsoft.Quantum/Workspaces/aq-hackathon-01",
  location="East US"
)

In [3]:
# create job_doc
filepath = ''
filename_job = 'minedoc.txt'

In [4]:
# # test - generating hash1

# # initialize a string
# x = np.random.rand(64)

# # create sha3-256 hash objects
# obj_sha3_256 = hashlib.sha3_256(x)

# # print in hexadecimal
# print("\nSHA3-256 Hash 1: ", obj_sha3_256.hexdigest())
# x_2 = obj_sha3_256.digest()
# x_3 = obj_sha3_256.hexdigest()

In [5]:
# converting SHA3 hash to a bit string to pass thru quantum process
def break_up_4bit_values(bin_string):
    # initialize hex string
    ini_string = bin_string
    scale = 16
    # printing initial string
    print ("Initial string", ini_string)
    # code to convert hex to binary
    res = bin(int(ini_string, scale)).zfill(8)
    # print the resultant string
    print ("Resultant string", str(res))
    array_4_bit_values = []
    i = 0
    while i < 64 : 
      four_bits = res[2+4*i:2+4*i+4]
      array_4_bit_values.append(four_bits)
      i = i + 1
    print(array_4_bit_values)
    return array_4_bit_values

In [6]:
# quantum-circuit task
def sim_quantum_operation(hash1_input):
    # importing Qiskit
    from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
    from qiskit import BasicAer, IBMQ, execute
    from qiskit.tools.monitor import job_monitor
    from azure.quantum.qiskit import AzureQuantumProvider

    # import basic plot tools
    from qiskit.tools.visualization import plot_histogram

    # useful additional packages 
    import numpy as np #library for scientific computing

    # from ibm_quantum_widgets import CircuitComposer
    from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
    from numpy import pi
    
    #input hash1 string
    fourbit_array = break_up_4bit_values(hash1_input)
    q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
    #print(four_bit_vector)
    
    #setting the quantum circuit: 5 qubits (and 5cbits)
    qreg_q = QuantumRegister(4, 'q')
    creg_c = ClassicalRegister(4, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    
    #circuit 6
    circ_layer = 1
    for i in range(circ_layer):
        circuit.rx(q_par[0]*pi/8, qreg_q[0])
        circuit.rx(q_par[1]*pi/8, qreg_q[1])
        circuit.rx(q_par[2]*pi/8, qreg_q[2])
        circuit.rx(q_par[3]*pi/8, qreg_q[3])
        circuit.rz(q_par[4]*pi/2, qreg_q[0])
        circuit.rz(q_par[5]*pi/8, qreg_q[1])
        circuit.rz(q_par[6]*pi/8, qreg_q[2])
        circuit.rz(q_par[7]*pi/8, qreg_q[3])
        circuit.crx(q_par[8]*pi/8, qreg_q[3], qreg_q[2])
        circuit.crx(q_par[9]*pi/8, qreg_q[3], qreg_q[1])
        circuit.crx(q_par[10]*pi/8, qreg_q[3], qreg_q[0])
        circuit.crx(q_par[11]*pi/8, qreg_q[2], qreg_q[3])
        circuit.crx(q_par[12]*pi/8, qreg_q[2], qreg_q[1])
        circuit.crx(q_par[13]*pi/8, qreg_q[2], qreg_q[0])
        circuit.crx(q_par[14]*pi/8, qreg_q[1], qreg_q[3])
        circuit.crx(q_par[15]*pi/8, qreg_q[1], qreg_q[2])
        circuit.crx(q_par[16]*pi/8, qreg_q[1], qreg_q[0])
        circuit.rz(q_par[17]*pi/8, qreg_q[3])
        circuit.rz(q_par[18]*pi/8, qreg_q[2])
        circuit.rz(q_par[19]*pi/8, qreg_q[0])
        circuit.rz(q_par[20]*pi/8, qreg_q[1])
        circuit.crx(q_par[21]*pi/8, qreg_q[0], qreg_q[3])
        circuit.crx(q_par[22]*pi/8, qreg_q[0], qreg_q[2])
        circuit.crx(q_par[23]*pi/8, qreg_q[0], qreg_q[1])
        circuit.rx(q_par[24]*pi/8, qreg_q[3])
        circuit.rx(q_par[25]*pi/8, qreg_q[2])
        circuit.rx(q_par[26]*pi/8, qreg_q[0])
        circuit.rx(q_par[27]*pi/8, qreg_q[1])
    
    #measurements of all qubits
    for i in range(len(qreg_q)):
        circuit.measure(qreg_q[i], creg_c[i]) 
    
    
    #Azure backends

    # print([backend.name() for backend in provider.backends()])

    simulator_backend = provider.get_backend("ionq.simulator")

    job = simulator_backend.run(circuit, shots=15000)
    job_id = job.id()
    print("Job id", job_id)

    job_monitor(job)

    result = job.result()
    type(result)
    print(result)
    counts = {format(n, "03b"): 0 for n in range(8)}
    counts.update(result.get_counts(circuit))
    print(counts)
    
    hist_filename = 'histogram' + job_id
    plot_histogram(counts, filename = hist_filename)

    #draw the quantum circuit
    draw_filename = 'q_circuit' + job_id
    circuit.draw(filename=draw_filename)

    #picking up the maximally probable state
    max_state = max(counts, key=counts.get)
    max_state256 = max_state

    for i in range(256 - len(max_state)):
        max_state256+='0' 
    
    with open(filepath+filename_job, 'w') as writefile:
        writefile.write(str(job_id) + ',' + str(max_state)+',')
    
    return max_state256 #4bit vector
    
    

In [7]:
MAX_NONCE = 100000000000

def qPoW(text,sim_QC):
    
    Initial = hashlib.sha3_256(text.encode("ascii")).hexdigest()
    if sim_QC == 0:
      Second = sim_quantum_operation(Initial)
    else: 
        Second = quantum_operation(Initial)
    
    Initial_Binary = " ".join(f"{ord(i):08b}" for i in Initial)
    Second_Binary = " ".join(f"{ord(i):08b}" for i in Second)
    print(Initial_Binary)
    print(Second_Binary)
    Third = xor(Initial_Binary,Second_Binary,256)
    print(Third)
    Fourth = hashlib.sha3_256(Third.encode("ascii")).hexdigest()
    print(Fourth)
    return Fourth

def mine(block_number, transactions, previous_hash, prefix_zeros,sim_QC):
    prefix_str = '0'*prefix_zeros
    for nonce in range(MAX_NONCE):
        text = str(block_number) + transactions + previous_hash + str(nonce)
        new_hash = qPoW(text,sim_QC)
        if new_hash.startswith(prefix_str):
            print(f"Yay! Successfully mined bitcoins with nonce value:{nonce}")
            return new_hash
    
    with open(filepath+filename_job, 'w') as writefile:
        writefile.write(str(nonce) + '\n')
    
    raise BaseException(f"Couldn't find correct has after trying {MAX_NONCE} times")

def Mine_One_Block(sim_QC, difficulty):

    if __name__=='__main__':
        transactions='''
      MSFT sent IONQ 300 qBTC
        '''
        difficulty= difficulty # try changing this to higher number and you will see it will take more time for mining as difficulty increases
        import time
        start = time.time()
        print("start mining")
        new_hash = mine(5,transactions,'0000000xa036944e29568d0cff17edbe038f81208fecf9a66be9a2b8321c6ec7', difficulty, sim_QC)
        total_time = str((time.time() - start))
        print(f"end mining. mining took: {total_time} seconds")
        print(new_hash)
        return

In [8]:
def xor(a, b, n):
    ans = ""
     
    # Loop to iterate over the
    # Binary Strings
    for i in range(n):
         
        # If the Character matches
        if (a[i] == b[i]):
            ans += "0"
        else:
            ans += "1"
    return ans
 

In [9]:
Mine_One_Block(0,1)

start mining
Initial string 5963915a0242b6f6748c5e981066f8d7413d18571c3cd451e249a18d72cf38f1
Resultant string 0b101100101100011100100010101101000000010010000101011011011110110011101001000110001011110100110000001000001100110111110001101011101000001001111010001100001010111000111000011110011010100010100011110001001001001101000011000110101110010110011110011100011110001
['1011', '0010', '1100', '0111', '0010', '0010', '1011', '0100', '0000', '0100', '1000', '0101', '0110', '1101', '1110', '1100', '1110', '1001', '0001', '1000', '1011', '1101', '0011', '0000', '0010', '0000', '1100', '1101', '1111', '0001', '1010', '1110', '1000', '0010', '0111', '1010', '0011', '0000', '1010', '1110', '0011', '1000', '0111', '1001', '1010', '1000', '1010', '0011', '1100', '0100', '1001', '0011', '0100', '0011', '0001', '1010', '1110', '0101', '1001', '1110', '0111', '0001', '1110', '001']
Job id 19ca9924-81ea-11ec-b084-448500f2e251
Job Status: job has successfully run
Result(backend_name='ionq.simulator', b

Job id 49f8c59c-81ea-11ec-8090-448500f2e251
Job Status: job has successfully run
Result(backend_name='ionq.simulator', backend_version='1', qobj_id='circuit-2', job_id='49f8c59c-81ea-11ec-8090-448500f2e251', success=True, results=[ExperimentResult(shots=15000, success=True, meas_level=2, data=ExperimentResultData(counts=defaultdict(<class 'int'>, {'0000': 33, '0001': 787, '0010': 373, '0011': 194, '0100': 1101, '0101': 388, '0110': 9051, '0111': 585, '1000': 277, '1001': 325, '1010': 43, '1011': 108, '1100': 320, '1101': 17, '1110': 107, '1111': 1283}), probabilities=defaultdict(<class 'int'>, {'0000': 0.002249018, '0001': 0.052509139, '0010': 0.024870748, '0011': 0.012971206, '0100': 0.073428978, '0101': 0.025901887, '0110': 0.603430096, '0111': 0.03905196, '1000': 0.01847902, '1001': 0.021689375, '1010': 0.002875416, '1011': 0.007247618, '1100': 0.021348859, '1101': 0.001188956, '1110': 0.007188868, '1111': 0.085568857})), header=QobjExperimentHeader(meas_map='[0, 1, 2, 3]', name='ci

Job id 7fe4ea3a-81ea-11ec-a85d-448500f2e251
Job Status: job has successfully run
Result(backend_name='ionq.simulator', backend_version='1', qobj_id='circuit-4', job_id='7fe4ea3a-81ea-11ec-a85d-448500f2e251', success=True, results=[ExperimentResult(shots=15000, success=True, meas_level=2, data=ExperimentResultData(counts=defaultdict(<class 'int'>, {'0000': 218, '0001': 423, '0010': 1833, '0011': 120, '0100': 209, '0101': 2753, '0110': 1043, '0111': 187, '1000': 474, '1001': 721, '1010': 41, '1011': 36, '1100': 432, '1101': 4933, '1110': 56, '1111': 1514}), probabilities=defaultdict(<class 'int'>, {'0000': 0.014568459, '0001': 0.02822826, '0010': 0.12220043, '0011': 0.00802696, '0100': 0.01399928, '0101': 0.183573707, '0110': 0.069554836, '0111': 0.012480032, '1000': 0.0316051, '1001': 0.048078714, '1010': 0.002785815, '1011': 0.002412047, '1100': 0.028848369, '1101': 0.328919168, '1110': 0.003750866, '1111': 0.100967957})), header=QobjExperimentHeader(meas_map='[0, 1, 2, 3]', name='circ

Job id af2899c2-81ea-11ec-887b-448500f2e251
Job Status: job has successfully run
Result(backend_name='ionq.simulator', backend_version='1', qobj_id='circuit-6', job_id='af2899c2-81ea-11ec-887b-448500f2e251', success=True, results=[ExperimentResult(shots=15000, success=True, meas_level=2, data=ExperimentResultData(counts=defaultdict(<class 'int'>, {'0000': 197, '0001': 40, '0010': 556, '0011': 245, '0100': 2133, '0101': 1210, '0110': 199, '0111': 7546, '1000': 129, '1001': 285, '1010': 144, '1011': 15, '1100': 1906, '1101': 163, '1110': 215, '1111': 8}), probabilities=defaultdict(<class 'int'>, {'0000': 0.013168738, '0001': 0.002721709, '0010': 0.037077512, '0011': 0.01635985, '0100': 0.142258788, '0101': 0.080726669, '0110': 0.013313516, '0111': 0.50312055, '1000': 0.008637267, '1001': 0.019020334, '1010': 0.009666565, '1011': 0.001010995, '1100': 0.127111035, '1101': 0.010890857, '1110': 0.014370981, '1111': 0.000544634})), header=QobjExperimentHeader(meas_map='[0, 1, 2, 3]', name='ci

Job id df72b0ac-81ea-11ec-a6c3-448500f2e251
Job Status: job has successfully run
Result(backend_name='ionq.simulator', backend_version='1', qobj_id='circuit-8', job_id='df72b0ac-81ea-11ec-a6c3-448500f2e251', success=True, results=[ExperimentResult(shots=15000, success=True, meas_level=2, data=ExperimentResultData(counts=defaultdict(<class 'int'>, {'0000': 149, '0001': 574, '0010': 92, '0011': 2615, '0100': 1070, '0101': 2828, '0110': 876, '0111': 634, '1000': 138, '1001': 128, '1010': 1894, '1011': 1496, '1100': 290, '1101': 664, '1110': 632, '1111': 912}), probabilities=defaultdict(<class 'int'>, {'0000': 0.009997383, '0001': 0.038303004, '0010': 0.006186813, '0011': 0.174342628, '0100': 0.071362853, '0101': 0.188548874, '0110': 0.058414107, '0111': 0.042316733, '1000': 0.009216917, '1001': 0.008540439, '1010': 0.12630788, '1011': 0.09974281, '1100': 0.019381844, '1101': 0.044325817, '1110': 0.042150749, '1111': 0.06086115})), header=QobjExperimentHeader(meas_map='[0, 1, 2, 3]', name=